# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

P. Sudarshan  ->  P. Sudarshan  |  ['P. Sudarshan']
M. Flock  ->  M. Flock  |  ['M. Flock']
R. P. Dubey  ->  R. P. Dubey  |  ['R. P. Dubey']
C. Fendt  ->  C. Fendt  |  ['C. Fendt']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Liu  ->  J. Liu  |  ['J. Liu']
M. McCaughrean  ->  M. McCaughrean  |  ['M. McCaughrean']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
S. Reyes  ->  S. Reyes  |  ['S. Reyes']
T. Henning  ->  T. Henning  |  ['T. Henning']


Arxiv has 71 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.15420


extracting tarball to tmp_2409.15420...

 done.


Found 99 bibliographic references in tmp_2409.15420/rad-project-dust.bbl.
Retrieving document from  https://arxiv.org/e-print/2409.15983


...

Found 67 bibliographic references in tmp_2409.15983/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2409.16028


extracting tarball to tmp_2409.16028... done.
Retrieving document from  https://arxiv.org/e-print/2409.16061
extracting tarball to tmp_2409.16061...

 done.


M. McCaughrean  ->  M. McCaughrean  |  ['M. McCaughrean']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
S. Reyes  ->  S. Reyes  |  ['S. Reyes']
T. Henning  ->  T. Henning  |  ['T. Henning']


Found 115 bibliographic references in tmp_2409.16061/HH211astroph.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.15420-b31b1b.svg)](https://arxiv.org/abs/2409.15420) | **Dusty substructures induced by planets in ALMA disks: how dust growth and dynamics changes the picture**  |
|| A. Ziampras, et al. -- incl., <mark>P. Sudarshan</mark>, <mark>M. Flock</mark> |
|*Appeared on*| *2024-09-25*|
|*Comments*| *15 pages, 15 figures, 2 tables; submitted to MNRAS, uploaded for visibility during PLUTO symposium; suggestions and comments welcome*|
|**Abstract**|            Protoplanetary disks exhibit a rich variety of substructure in millimeter continuum emission, often attributed to unseen planets. As these planets carve gaps in the gas, dust particles can accumulate in the resulting pressure bumps, forming bright features in the dust continuum. We investigate the role of dust dynamics in the gap-opening process with 2D radiation hydrodynamics simulations of planet--disk interaction and a two-population dust component modeled as a pressureless fluid. We consider the opacity feedback and backreaction due to drag forces as mm grains accumulate in pressure bumps at different stages of dust growth. We find that dust dynamics can significantly affect the resulting substructure driven by the quasi-thermal-mass planet with $M_p/M_\star=10^{-4}$. Opacity feedback causes nonaxisymmetric features to become more compact in azimuth, whereas the drag-induced backreaction tends to dissolve nonaxisymmetries. For our fiducial model, this results in multiple concentric rings of dust rather than the expected vortices and corotating dust clumps found in models without dust feedback. A higher coagulation fraction disproportionately enhances the effect of dust opacity feedback, favoring the formation of crescents rather than rings. Our results suggest that turbulent diffusion is not always necessary to explain the rarity of observed nonaxisymmetric features, and that incorporating dust dynamics is vital for interpreting the observed substructure in protoplanetary disks. We also describe and test the implementation of the publicly-available dust fluid module in the PLUTO code.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.15983-b31b1b.svg)](https://arxiv.org/abs/2409.15983) | **Particles in Relativistic MHD Jets II: Bridging Jet Dynamics with Multi-waveband Non-Thermal Emission Signatures**  |
|| <mark>R. P. Dubey</mark>, <mark>C. Fendt</mark>, B. Vaidya |
|*Appeared on*| *2024-09-25*|
|*Comments*| *Submitted to The Astrophysical Journal (ApJ)*|
|**Abstract**|            Relativistic magnetized jets, originating near black holes, are observed to exhibit sub-structured flows. In this study, we present synthetic synchrotron emission signatures for different lines of sight and frequencies, derived from three-dimensional relativistic magneto-hydrodynamic simulations of pc-scale AGN jets. These simulations apply different injection nozzles, injecting steady, variable, and precessing jets. Extending our previous study, here, we have developed a bridge to connect jet dynamics and particle acceleration within relativistic shocks with non-thermal radiation dominant in jets. The emission is derived from Lagrangian particles - injected into the jet and following the fluid - accelerated through diffusive shock acceleration and subsequently cooled by emitting energy via synchrotron and inverse-Compton processes. Overall, the different shocks structures lead to the formation of numerous localized emission patterns - interpreted as jet knots. These knot patterns can fade or flare, also as a consequence of merging or Doppler boosting, leading to jet variability. We find knots with high-enough pattern speed supposed to be visible as superluminal motion <~5c. Synchrotron spectra of all jets reveal double-humped structures, reflecting multiple electron populations characterized by the nature of underlying shock and their age. The precessing jet is the most powerful emitter, featuring a spectrum flatter than the steady and the variable jet. The emission, although essentially governed by the acceleration through shocks, depends on the cooling history of the particle as well. Overall, the continuous re-acceleration of electrons through shocks along the jet we found, is an essential prerequisite for observing extended jet emission over large time-scales and length-scales.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.16061-b31b1b.svg)](https://arxiv.org/abs/2409.16061) | **JWST Observations of Young protoStars (JOYS). HH 211: the textbook case of a protostellar jet and outflow**  |
|| A. C. o. Garatti, et al. -- incl., <mark>M. McCaughrean</mark>, <mark>H. Beuther</mark>, <mark>S. Reyes</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-09-25*|
|*Comments*| *Paper accepted in A&A for publication*|
|**Abstract**|            We use the James Webb Space Telescope (JWST) and its Mid-Infrared Instrument (MIRI) (5-28 um), to study the embedded HH 211 flow. We map a 0.95'x0.22' region, covering the full extent of the blue-shifted lobe, the central protostellar region, and a small portion of the red-shifted lobe. The jet driving source is not detected even at the longest mid-IR wavelengths. The overall morphology of the flow consists of a highly collimated jet, mostly molecular (H2, HD) with an inner atomic ([FeI], [FeII], [SI], [NiII]) structure. The jet shocks the ambient medium, producing several large bow-shocks, rich in forbidden atomic and molecular lines, and is driving an H2 molecular outflow, mostly traced by low-J, v=0 transitions. Moreover, 0-0 S(1) uncollimated emission is also detected down to 2"-3" (~650-1000 au) from the source, tracing a cold (T=200-400 K), less dense and poorly collimated molecular wind. The atomic jet ([FeII] at 26 um) is detected down to ~130 au from source, whereas the lack of H2 emission close to the source is likely due to the large visual extinction. Dust continuum-emission is detected at the terminal bow-shocks, and in the blue- and red-shifted jet, being likely dust lifted from the disk. The jet shows an onion-like structure, with layers of different size, velocity, temperature, and chemical composition. Moreover, moving from the inner jet to the outer bow-shocks, different physical, kinematic and excitation conditions for both molecular and atomic gas are observed. The jet mass-flux rate, momentum, and momentum flux of the warm H2 component are up to one order of magnitude higher than those inferred from the atomic jet component. Our findings indicate that the warm H2 component is the primary mover of the outflow, namely it is the most significant dynamical component of the jet, in contrast to jets from more evolved YSOs, where the atomic component is dominant.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.16028-b31b1b.svg)](https://arxiv.org/abs/2409.16028) | **Chromospheric modeling of the active M3V star G 80-21 with RH1.5D**  |
|| S. Liu, et al. -- incl., <mark>J. Shi</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2024-09-25*|
|*Comments*| *Accepted for publication in The Astrophysical Journal*|
|**Abstract**|            This study investigates the active regions of the M3.0V star G 80-21 using the observed data from the CARMENES project with synthetic spectra generated by the RH1.5D radiative transfer code. The CARMENES project aims to search for exoplanets around M dwarfs using high-resolution near-infrared and optical echelle spectrographs. By comparing the observed data and models for the chromospheric lines of H$_\alpha$ and the bluest Ca II infrared triplet line, we obtain the best-fit models for this star. The optimal fitting for the observed spectrum of G 80-21 is achieved by employing two active areas in conjunction with an inactive regions, with a calcium abundance of [Ca/H] = $-$0.4. This combination successfully fits all the observed data across varying ratios. The minor active component consistently comprises approximately 18\% of the total (ranging from 14\% to 20\%), which suggests that the minor active component is likely located in the polar regions. Meanwhile, the major active component occupies a variable proportion, ranging from 51\% to 82\%. Our method allows for the determination of the structure and size of stellar chromospheric active regions by analyzing high-resolution observed spectra.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.15420/./compare-X-1D.png', 'tmp_2409.15420/./compare-X.png', 'tmp_2409.15420/./kappa-temperature.png']
copying  tmp_2409.15420/./compare-X-1D.png to _build/html/
copying  tmp_2409.15420/./compare-X.png to _build/html/
copying  tmp_2409.15420/./kappa-temperature.png to _build/html/
exported in  _build/html/2409.15420.md
    + _build/html/tmp_2409.15420/./compare-X-1D.png
    + _build/html/tmp_2409.15420/./compare-X.png
    + _build/html/tmp_2409.15420/./kappa-temperature.png
found figures ['tmp_2409.15983/./dubey_fig9.png', 'tmp_2409.15983/./dubey_fig3a.png', 'tmp_2409.15983/./dubey_fig3b.png', 'tmp_2409.15983/./dubey_fig3c.png', 'tmp_2409.15983/./dubey_fig4.png']
copying  tmp_2409.15983/./dubey_fig9.png to _build/html/
copying  tmp_2409.15983/./dubey_fig3a.png to _build/html/
copying  tmp_2409.15983/./dubey_fig3b.png to _build/html/
copying  tmp_2409.15983/./dubey_fig3c.png to _build/html/
copying  tmp_2409.15983/./dubey_fig4.png to _build/html/
exported in  _b

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tensor}[1]{\overline{\textbf{#1}}}$
$\newcommand{\tensorGR}[1]{\overline{\bm{{#1}}}}$
$\newcommand{\DP}[2]{\frac{\partial{#1}}{\partial{#2}}}$
$\newcommand{\D}[2]{\frac{\text{d}{#1}}{\text{d}{#2}}}$
$\newcommand{\ep}{e_\mathrm{p}}$
$\newcommand{\ap}{a_\mathrm{p}}$
$\newcommand{\G}{\text{G}}$
$\newcommand{\Mstar}{M_\star}$
$\newcommand{\Lstar}{L_\star}$
$\newcommand{\Rp}{R_\mathrm{p}}$
$\newcommand{\Mp}{M_\mathrm{p}}$
$\newcommand{\hp}{h_\mathrm{p}}$
$\newcommand{\Hp}{H_\mathrm{p}}$
$\newcommand{\Tp}{T_\mathrm{p}}$
$\newcommand{\Pp}{P_\mathrm{p}}$
$\newcommand{\Tb}{T_\mathrm{b}}$
$\newcommand{\Mth}{M_\mathrm{th}}$
$\newcommand{\Msun}{\mathrm{M}_\odot}$
$\newcommand{\Lsun}{\mathrm{L}_\odot}$
$\newcommand{\Mjup}{\mathrm{M}_\mathrm{J}}$
$\newcommand{\Rgas}{\mathcal{R}}$
$\newcommand{\cs}{c_\mathrm{s}}$
$\newcommand{\csiso}{c_\mathrm{s,iso}}$
$\newcommand{\csadb}{c_\mathrm{s}^\mathrm{ad}}$
$\newcommand{\OmegaK}{\Omega_\mathrm{K}}$
$\newcommand{\vK}{u_\mathrm{K}}$
$\newcommand{\mean}[1]{\langle{#1} \rangle}$
$\newcommand{\tauR}{\tau_\mathrm{R}}$
$\newcommand{\tauP}{\tau_\mathrm{P}}$
$\newcommand{\tauReff}{\tau_\mathrm{R}^\mathrm{eff}}$
$\newcommand{\tauPeff}{\tau_\mathrm{P}^\mathrm{eff}}$
$\newcommand{\taueff}{\tau_\mathrm{eff}}$
$\newcommand{\kappaR}{\kappa_\mathrm{R}}$
$\newcommand{\kappaP}{\kappa_\mathrm{P}}$
$\newcommand{\cv}{c_\mathrm{v}}$
$\newcommand{\rhomid}{\rho_\mathrm{mid}}$
$\newcommand{\sigmaSB}{\sigma_\mathrm{SB}}$
$\newcommand{\vel}{\bm{u}}$
$\newcommand{\xh}{{x}_\mathrm{h}}$
$\newcommand{\varpih}{{\varpi}_\mathrm{h}}$
$\newcommand{\tcool}{t_\mathrm{cool}}$
$\newcommand{\bcool}{\beta_\mathrm{cool}}$
$\newcommand{\bsurf}{\beta_\mathrm{surf}}$
$\newcommand{\bmid}{\beta_\mathrm{mid}}$
$\newcommand{\btot}{\beta_\mathrm{tot}}$
$\newcommand{\bcoll}{\beta_\mathrm{coll}}$
$\newcommand{\bbuoy}{\beta_\mathrm{buoy}}$
$\newcommand{\bdiff}{\beta_\text{diff}}$
$\newcommand{\bfld}{\beta_\mathrm{FLD}}$
$\newcommand{\Qvisc}{Q_\mathrm{visc}}$
$\newcommand{\Qcool}{Q_\mathrm{cool}}$
$\newcommand{\Qirr}{Q_\mathrm{irr}}$
$\newcommand{\Qrad}{Q_\mathrm{rad}}$
$\newcommand{\Qrelax}{Q_\mathrm{relax}}$
$\newcommand{\Erad}{E_\mathrm{rad}}$
$\newcommand{\aR}{a_\mathrm{R}}$
$\newcommand{\lrad}{l_\mathrm{rad}}$
$\newcommand{\Sigmag}{\Sigma_\mathrm{g}}$
$\newcommand{\Sigmad}{\Sigma_\mathrm{d}}$
$\newcommand{\velg}{\vel_\mathrm{g}}$
$\newcommand{\veld}{\vel_\mathrm{d}}$
$\newcommand{\veli}{\vel_\mathrm{i}}$
$\newcommand{\velj}{\vel_\mathrm{j}}$
$\newcommand{\St}{\mathrm{St}}$
$\newcommand{\ts}{t_\mathrm{s}}$
$\newcommand{\ad}{a_\mathrm{d}}$
$\newcommand{\brhod}{\bar{\rho}_\mathrm{d}}$
$\newcommand{\sd}{s_\mathrm{d}}$
$\newcommand{\md}{m_\mathrm{d}}$
$\newcommand{\pluto}{\texttt{PLUTO}}$
$\newcommand{\dw}{\texttt{DUSTYWAVE}}$
$\newcommand{\fargo}{{\texttt{FARGO3D}}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Dusty substructures induced by planets in ALMA disks:\\how dust growth and dynamics changes the picture

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.15420-b31b1b.svg)](https://arxiv.org/abs/2409.15420)<mark>Appeared on: 2024-09-25</mark> -  _15 pages, 15 figures, 2 tables; submitted to MNRAS, uploaded for visibility during PLUTO symposium; suggestions and comments welcome_

</div>
<div id="authors">

A. Ziampras, et al. -- incl., <mark>P. Sudarshan</mark>, <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** Protoplanetary disks exhibit a rich variety of substructure in millimeter continuum emission, often attributed to unseen planets. As these planets carve gaps in the gas, dust particles can accumulate in the resulting pressure bumps, forming bright features in the dust continuum. We investigate the role of dust dynamics in the gap-opening process with 2D radiation hydrodynamics simulations of planet--disk interaction and a two-population dust component modeled as a pressureless fluid. We consider the opacity feedback and backreaction due to drag forces as mm grains accumulate in pressure bumps at different stages of dust growth. We find that dust dynamics can significantly affect the resulting substructure driven by the quasi-thermal-mass planet with $\Mp/\Mstar=10^{-4}$ . Opacity feedback causes nonaxisymmetric features to become more compact in azimuth, whereas the drag-induced backreaction tends to dissolve nonaxisymmetries. For our fiducial model, this results in multiple concentric rings of dust rather than the expected vortices and corotating dust clumps found in models without dust feedback. A higher coagulation fraction disproportionately enhances the effect of dust opacity feedback, favoring the formation of crescents rather than rings. Our results suggest that turbulent diffusion is not always necessary to explain the rarity of observed nonaxisymmetric features, and that incorporating dust dynamics is vital for interpreting the observed substructure in protoplanetary disks. We also describe and test the implementation of the publicly-available dust fluid module in the $\pluto$ code.

</div>

<div id="div_fig1">

<img src="tmp_2409.15420/./compare-X-1D.png" alt="Fig5" width="100%"/>

**Figure 5. -** Azimuthally averaged gas surface density (top), dust surface density (middle), and dust brightness temperature at 1.25 mm (bottom) for models with different coagulation fractions $X$. The secondary gap in the gas surface density is shallower for smaller $X$(i.e., more mass in small grains), due to the longer cooling timescale in the inner disk. Substructure in the dust is less pronounced for $X=0.01$, due to the well-coupled small grains representing most of the dust mass. (*fig:compare-X-1D*)

</div>
<div id="div_fig2">

<img src="tmp_2409.15420/./compare-X.png" alt="Fig7" width="100%"/>

**Figure 7. -** Brightness temperature heatmaps at 1.25 mm for models with different coagulation fractions $X$. The locally isothermal model (panel *a*) shows a fainter ring than its radiative counterpart (*b*) near the inner gap edge and an azimuthally extended bright feature inside the corotating region. The model with $X=0.01$(*d*) is overall brighter and shows less pronounced features, as the bulk of the dust is perfectly coupled to the gas. (*fig:compare-X*)

</div>
<div id="div_fig3">

<img src="tmp_2409.15420/./kappa-temperature.png" alt="Fig2" width="100%"/>

**Figure 2. -** Rosseland and Planck mean opacities as a function of temperature for small (blue) and big (orange) grains. Small grains follow a power law $\propto T^{1.6}$, while big grains have a roughly constant opacity. (*fig:opacity-temperature*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.15420"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vecc}{\textbf{\textit }}$</div>



<div id="title">

# Particles in Relativistic MHD Jets II: Bridging Jet Dynamics with Multi-waveband Non-Thermal Emission Signatures

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.15983-b31b1b.svg)](https://arxiv.org/abs/2409.15983)<mark>Appeared on: 2024-09-25</mark> -  _Submitted to The Astrophysical Journal (ApJ)_

</div>
<div id="authors">

<mark>R. P. Dubey</mark>, <mark>C. Fendt</mark>, B. Vaidya

</div>
<div id="abstract">

**Abstract:** Relativistic magnetized jets, originating near black holes, are observed to exhibit sub-structured flows.In this study, we present synthetic synchrotron emission signatures for different lines of sight and frequencies,derived from three-dimensional relativistic magneto-hydrodynamic simulations of pc-scale AGN jets.These simulations apply different injection nozzles, injecting steady, variable, and precessing jets.Extending our previous study, here, we have developed a bridge to connect jet dynamics and particle acceleration withinrelativistic shocks with non-thermal radiation dominant in jets.The emission is derived from Lagrangian particles - injected into the jet and following the fluid - accelerated through diffusiveshock acceleration and subsequently cooled by emitting energy via synchrotron and inverse-Compton processes.Overall, the different shocks structures lead to the formation of numerous localized emission patterns - interpreted as jet knots.These knot patterns can fade or flare, also as a consequence of merging or Doppler boosting, leading to jet variability.We find knots with high-enough pattern speed supposed to be visible as superluminal motion $\lesssim 5c$ .Synchrotron spectra of all jets reveal double-humped structures, reflecting multiple electron populations characterized by the natureof underlying shock and their age.The precessing jet is the most powerful emitter, featuring a spectrum flatter than the steady and the variable jet.The emission, although essentially governed by the acceleration through shocks, depends on the cooling history of the particle as well.Overall, the continuous re-acceleration of electrons through shocks along the jet we found,is an essential prerequisite for observing extended jet emission over large time-scales and length-scales.

</div>

<div id="div_fig1">

<img src="tmp_2409.15983/./dubey_fig9.png" alt="Fig10" width="100%"/>

**Figure 10. -** Radiation spectra from simulations of a steady jet (left), a variable jet (center), and a precessing jet (right),
    at time $t=50$(in code units).
    In all three panels, the blue, orange and green curves represent the inclination
    $\hat{_**n**_}_{\rm los} = 90^\circ$, $45^\circ$, and $10^\circ$ respectively.
    Fluxes are integrated over the whole computational domain, similar to an unresolved * core* emission. (*fig:spc_t50*)

</div>
<div id="div_fig2">

<img src="tmp_2409.15983/./dubey_fig3a.png" alt="Fig4.1" width="33%"/><img src="tmp_2409.15983/./dubey_fig3b.png" alt="Fig4.2" width="33%"/><img src="tmp_2409.15983/./dubey_fig3c.png" alt="Fig4.3" width="33%"/>

**Figure 4. -** 2D distribution of specific intensity $I_{\nu}$(in $\log$ scale), blurred with a Gaussian beam, in the plane of the sky at frequency $\nu = 1$ GHz and time $t=50$(in code units)
    after integrating the specific emissivity $J_{\nu}$ along the line of sight.
    Shown are the maps for
    steady (left), variable (center), and precessing (right) jets with inclination $\hat{_**n**_}_{\rm los} = 90^\circ$(top panel), $45^\circ$(middle panel) and $10^\circ$(bottom panel), respectively.
    The colorbars indicate the level of specific intensity in $\rm{erg  s^{-1}cm^{-2}str^{-1}Hz^{-1}}$(in $\log$ scale).
    The black contours indicate 0.01, 0.1, 1, 5, and 10 per cent of maximum intensity in all panels.
    Additionally, contours at $10^{-5}$ of the maximum intensity are added in the middle panel
    for better visualisation.
     (*fig:blur_intensity_1GHZ*)

</div>
<div id="div_fig3">

<img src="tmp_2409.15983/./dubey_fig4.png" alt="Fig5" width="100%"/>

**Figure 5. -** Mock radiant intensity ($\nu I_{\nu}$) distribution (in $\log$ scale) in the plane of the sky  for the steady jet * Std90* at time $t=50$
    and frequency $\nu = 1$ GHz (left), $9\times 10^4$ GHz (center), and $10^9$ GHz (right), lying in the radio, optical and X-ray regime, respectively.
    The colorbars indicate the levels of radiant intensity in $10^9 \rm{erg s^{-1}cm^{-2}str^{-1}}$(in $\log$ scale).  (*fig:multi_int*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.15983"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\lsun}{\mbox{L}_\odot}$
$\newcommand{\msun}{\mbox{M}_\odot}$
$\newcommand{\macc}{\dot{M}_{\rm acc} }$
$\newcommand{\FeII}{[\ion{Fe}{ii}] }$
$\newcommand{\FeIIp}{[\ion{Fe}{ii}]}$
$\newcommand{\FeI}{[\ion{Fe}{i}] }$
$\newcommand{\FeIp}{[\ion{Fe}{i}]}$
$\newcommand{\TiII}{[\ion{Ti}{ii}]}$
$\newcommand{\SII}{[\ion{S}{ii}]}$
$\newcommand{\SIIp}{[\ion{S}{ii}] }$
$\newcommand{\OI}{[\ion{O}{i}] }$
$\newcommand{\OIp}{[\ion{O}{i}]}$
$\newcommand{\PII}{[\ion{P}{ii}]}$
$\newcommand{\NiI}{[\ion{Ni}{i}] }$
$\newcommand{\NiII}{[\ion{Ni}{ii}] }$
$\newcommand{\NiIIp}{[\ion{Ni}{ii}]}$
$\newcommand{\NIp}{\ion{N}{i}}$
$\newcommand{\CaIIp}{\ion{Ca}{ii}}$
$\newcommand{\PI}{[\ion{P}{i}]}$
$\newcommand{\NeII}{[\ion{Ne}{ii}] }$
$\newcommand{\NeIIp}{[\ion{Ne}{ii}]}$
$\newcommand{\CoII}{[\ion{Co}{ii}] }$
$\newcommand{\CoIIp}{[\ion{Co}{ii}]}$
$\newcommand{\ClI}{[\ion{Cl}{i}] }$
$\newcommand{\ClIp}{[\ion{Cl}{i}]}$
$\newcommand{\ClII}{[\ion{Cl}{ii}] }$
$\newcommand{\ClIIp}{[\ion{Cl}{ii}]}$
$\newcommand{\CIp}{\ion{C}{i}}$
$\newcommand{\HeI}{\ion{He}{i}}$
$\newcommand{\MgIp}{\ion{Mg}{i}}$
$\newcommand{\MgIIp}{\ion{Mg}{ii}}$
$\newcommand{\NaI}{\ion{Na}{i}}$
$\newcommand{\HI}{\ion{H}{i} }$
$\newcommand{\HIp}{\ion{H}{i}}$
$\newcommand{\SI}{[\ion{S}{i}] }$
$\newcommand{\SiII}{[\ion{Si}{ii}]}$
$\newcommand{\SIp}{[\ion{S}{i}]}$
$\newcommand{\SIII}{[\ion{S}{iii}] }$
$\newcommand{\SIIIp}{[\ion{S}{iii}]}$
$\newcommand{\ArII}{[\ion{Ar}{ii}] }$
$\newcommand{\ArIIp}{[\ion{Ar}{ii}]}$
$\newcommand{\brg}{Br\gamma}$
$\newcommand{\pab}{Pa\beta}$
$\newcommand{\lacc}{L_{acc}}$
$\newcommand{\lbol}{L_{bol}}$
$\newcommand{\mjet}{\dot{M}_{\rm jet}}$
$\newcommand{\mh}{\dot{M}_{H_2}}$
$\newcommand{\Ne}{n_e}$
$\newcommand{\h}{H_2}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\um}{\mum}$
$\newcommand{\lam}{\lambda}$
$\newcommand{\msyr}{M_{\sun} yr^{-1}}$
$\newcommand{\Av}{A_V }$
$\newcommand{\rsun}{R_{\sun}}$
$\newcommand{\cm}{cm^{-3}}$
$\newcommand{\ergscm}{erg s^{-1} cm^{-2}}$</div>



<div id="title">

# JWST Observations of Young protoStars (JOYS): HH 211: the textbook case of a protostellar jet and outflow

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.16061-b31b1b.svg)](https://arxiv.org/abs/2409.16061)<mark>Appeared on: 2024-09-25</mark> -  _Paper accepted in A&A for publication_

</div>
<div id="authors">

A. C. o. Garatti, et al. -- incl., <mark>M. McCaughrean</mark>, <mark>H. Beuther</mark>, <mark>S. Reyes</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Due to the high visual extinction and lack of sensitive mid-IR telescopes, the origin and properties of outflows and jets from embedded Class 0 protostars are still poorly constrained. We aim at characterising the physical, kinematic, and dynamical properties of the $\object{HH 211}$ jet and outflow, one of the youngest protostellar flows. We use the $_ James Webb Space Telescope_$ (JWST) and its Mid-Infrared Instrument (MIRI) in the 5--28 $\mu$ m range, to study the embedded $\object{HH 211}$ flow. We map a 0 $\farcm$ 95 $\times$ 0 $\farcm$ 22 region, covering the full extent of the blue-shifted lobe, the central protostellar region, and a small portion of the red-shifted lobe. We extract spectra along the jet and outflow and construct line and excitation maps of both atomic and molecular lines. Additional JWST NIRCam $H_2$ narrow-band images (at 2.122 and 3.235 $\mu$ m) provide a visual-extinction map of the whole flow, and are used to deredden our data. The jet driving source is not detected even at the longest mid-IR wavelengths. The overall morphology of the flow consists of a highly collimated jet, mostly molecular ($H_2$ , HD) with an inner atomic ( $\FeIp$ , $\FeIIp$ , $\SIp$ , $\NiIIp$ ) structure. The jet shocks the ambient medium, producing several large bow-shocks, rich in forbidden atomic ( $\FeIIp$ , $\SIp$ , $\NiIIp$ , $\ClIp$ , $\ClIIp$ , $\ArIIp$ , $\CoIIp$ , $\NeIIp$ , $\SIIIp$ ) and molecular lines ($H_2$ , HD, CO, OH, $H_2$ O, $CO_2$ , HCO $^+$ ), and is driving an $H_2$ molecular outflow, mostly traced by low- $J$ , $v=0$ transitions.   Moreover, $H_2$ 0-0 S(1) uncollimated emission is also detected down to 2 $\arcsec$ --3 $\arcsec$ ( $\sim$ 650--1000 au) from the source, tracing a cold ( $T$ =200--400 K), less dense and poorly collimated molecular wind. Two $H_2$ components (warm, $T$ =300--1000 K, and hot, $T$ =1000--3500 K) are detected along the jet and outflow.   The atomic jet ( $\FeII$ at 26 $\mu$ m) is detected down to $\sim$ 130 au from source, whereas the lack of $H_2$ emission (at 17 $\mu$ m) close to the source is likely due to the large visual extinction ( $\Av$ $>$ 80 mag).   Dust continuum-emission is detected at the terminal bow-shocks, and in the blue- and red-shifted jet, being likely dust lifted from the disk. The jet shows an onion-like structure, with layers of different size, velocity, temperature, and chemical composition.   Moreover, moving from the inner jet to the outer bow-shocks, different physical, kinematic and excitation conditions for both molecular and atomic gas are observed. The jet mass-flux rate, momentum, and momentum flux of the warm $H_2$ component are up to one order of magnitude higher than those inferred from the atomic jet component.   Our findings indicate that the warm $H_2$ component is the primary mover of the outflow, namely it is the most significant dynamical component of the jet, in contrast to jets from more evolved YSOs, where the atomic component is dominant.

</div>

<div id="div_fig1">

<img src="tmp_2409.16061/./figures/tricolor_3.jpg" alt="Fig19" width="100%"/>

**Figure 19. -** Tricolour MIRI-MRS map of $H_2$ 0-0 S(7) (at 5.5 $\mu$m, in blue), $H_2$ 0-0 S(1) (at 17 $\mu$m, in green), and $\FeII$(at 26 $\mu$m, in red) emission lines. The white circle marks the position of the ALMA mm continuum source. Black and white contours indicate the position (on the blue- and red-shifted lobe side, respectively) of continuum emission integrated between 25.3 and 25.9 $\mu$m (displayed contours are at 3, 5, and 50 $\sigma$; 1 $\sigma$=4 MJy sr$^{-1}$). Knots and bow-shocks (BS) showing continuum emission are indicated. (*fig:MIRI-tricolor*)

</div>
<div id="div_fig2">

<img src="tmp_2409.16061/./figures/HD23um_astro.png" alt="Fig12.1" width="25%"/><img src="tmp_2409.16061/./figures/CO.png" alt="Fig12.2" width="25%"/><img src="tmp_2409.16061/./figures/NiII.png" alt="Fig12.3" width="25%"/><img src="tmp_2409.16061/./figures/ClI.png" alt="Fig12.4" width="25%"/>

**Figure 12. -** 
   Additional line maps of some bright features detected along HH 211. From top to bottom: HD 0-0 R(4) at 23.03 $\mu$m, CO (1-0) lines from 4.96 to 4.98 $\mu$m, $\NiI$I at 6.64 $\mu$m, $\ClI$ at 11.33 $\mu$m. The magenta circle shows the position of the ALMA mm continuum source. Integrated flux is in mJy pixel$^{-1}$.  (*fig:all_maps*)

</div>
<div id="div_fig3">

<img src="tmp_2409.16061/./figures/v_map_SI25_clean.jpg" alt="Fig15.1" width="33%"/><img src="tmp_2409.16061/./figures/v_map_FeI._clean.jpg" alt="Fig15.2" width="33%"/><img src="tmp_2409.16061/./figures/v_map_FeII17_clean3.jpg" alt="Fig15.3" width="33%"/>

**Figure 15. -** $\SI$ at 25 $\mu$m (top panel) and $\FeI$ at 24 $\mu$m (middle panel) and $\FeII$ at 17.9 $\mu$m (bottom panel) velocity maps. Black contours in the top and bottom panels show the integrated continuum-subtracted line intensity at 5$\sigma$(0.8 mJy pixel$^{-1}$). Bottom panel black contours are at 5, 50, 100, and 500 $\sigma$(namely, 0.5, 5, 10, and 50 mJy pixel$^{-1}$
    Only velocities within a 5$\sigma$ threshold are plotted. The magenta circle shows the position of the ALMA mm continuum source. (*fig:atomic_vmap*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.16061"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

194  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
